In [ ]:
!pip install pydicom

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd
import cv2
import pydicom
import random
from collections import Counter, defaultdict
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.models as models
from torch.utils.data import Subset
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from torch.nn.utils import clip_grad_norm_

In [ ]:
torch.cuda.empty_cache()

In [ ]:
class Interpolate():
    
    def __init__(self, target_num = 22):
        self.target_num = target_num
    
    def __call__(self, slices): # (3, XX, 512, 512)
        slices = slices.unsqueeze(dim = 0) # (1, 3, XX, 512, 512)
        slices = F.interpolate(
                  slices, 
                  mode = 'trilinear',
                  size = (self.target_num, slices.shape[-2], slices.shape[-1])) # (1, 3, 22, 512, 512)
        return slices.squeeze() # (slices.squeeze(): (3, 22, 512, 512))

# transform
all_train_transform = [
    Interpolate(target_num = 22)
]

train_transform = transforms.Compose(all_train_transform)

In [ ]:
class MRI2DDataset(Dataset):
    def __init__(self, df, transforms = None):
        
        root = "/kaggle/input/hw3-data/hwk03_data/hwk03_data"
        self.ids = np.array(df["ID"])
        self.labels = torch.from_numpy(np.array(df["Disease"]))
        self.transforms = transforms
        
        paths = []
        for ID in np.array(df["ID"]):
            path = os.path.join(root, "DICOM", str(ID).split('.')[0].zfill(7))
            paths.append(path)
        
        self.images = []
        
        for path in paths:
            all_slices = []
            T1_root = os.path.join(root, "DICOM", path, "T1")
            for filename in sorted(os.listdir(T1_root), key = lambda s: int(pydicom.dcmread(os.path.join(T1_root, s)).InstanceNumber)):
                T1_image = []
                T2_image = []
                
                # T1 images
                img = pydicom.dcmread(os.path.join(T1_root, filename))
                ww = img.WindowWidth
                wc = img.WindowCenter
                highest_visible_value = (ww + 2 * wc) / 2
                lowest_visible_value = highest_visible_value - ww
                
                img = img.pixel_array
                img = np.clip(img, lowest_visible_value, highest_visible_value)
                img = 255 * (img - lowest_visible_value) / ww
                img = cv2.resize(img, (512, 512))
                
                T1_image.append(img)
                
                # T2 images
                img = pydicom.dcmread(os.path.join(root, "DICOM", path, "T2", filename))
                ww = img.WindowWidth
                wc = img.WindowCenter
                highest_visible_value = (ww + 2 * wc) / 2
                lowest_visible_value = highest_visible_value - ww
                
                img = img.pixel_array
                img = np.clip(img, lowest_visible_value, highest_visible_value)
                img = 255 * (img - lowest_visible_value) / ww
                img = cv2.resize(img, (512, 512))
                
                T2_image.append(img)
                
                # T1T2 images
                T1T2_image = (T1_image[0] + T2_image[0]) / 2
                T1T2_image = torch.tensor(T1T2_image)
                
                T1_image = torch.tensor(T1_image[0])
                T2_image = torch.tensor(T2_image[0])
                
                all_slices.append(torch.stack((T1_image, T2_image, T1T2_image), dim = 0))
                
            slices = torch.stack(all_slices, dim = 1)
            
            if self.transforms:
                slices = self.transforms(slices) 
                slices = torch.transpose(slices, 0, 1) # (22, 3, 512, 512)
            
            image = []
            for i in range(len(slices)):
                image.append(slices[i].squeeze().float()) 
            self.images.append(image)
                
            
    def __getitem__(self, index):
        images = self.images[index]
        label = self.labels[index]
        
        return images, label
    
    def __len__(self):
        return(len(self.ids))

In [ ]:
class config:
    
    root = '/kaggle/input/hw3-data/hwk03_data/hwk03_data'
    valid_prob = 0.2
    
    batch_size_early = 4
    lr_early = 1e-4
    epochs_early= 100
    weight_decay_early = 1e-3
    
 
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    seed = 42
    
print(config.device)

In [ ]:
def seed_everything(seed):
    # Set Python random seed
    random.seed(seed)
    
    # Set NumPy random seed
    np.random.seed(seed)
    
    # Set PyTorch random seed for CPU and GPU
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    
    # Set PyTorch deterministic operations for cudnn backend
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def evaluator(preds, gts):
    preds = preds.cpu().numpy() if isinstance(preds, torch.Tensor) else preds
    gts = gts.cpu().numpy() if isinstance(gts, torch.Tensor) else gts
    acc = accuracy_score(preds, gts)
    f1 = f1_score(preds, gts, average="macro")

    return acc, f1


def train_one_epoch(model, train_loader, optimizer, scheduler, criterion, device):
    model.train()
    train_loss = 0.0
    predictions, ground_truths = [], []

    for batch_idx, (images, labels) in enumerate(train_loader):
        # 將數據移動到指定的設備
        
        images = [img.to(device) for img in images]
        labels = labels.to(device)
        #print("Input shape:", images[0].shape)
        #print("Labels shape:", labels.shape)
        

        optimizer.zero_grad()
        logits = model(images)

        loss = criterion(logits, labels)
        loss.backward()

        optimizer.step()
        scheduler.step()

        train_loss += loss.item()
        preds = torch.argmax(logits, dim=1)

        predictions.append(preds)
        ground_truths.append(labels)

    train_loss /= len(train_loader)

    predictions = torch.cat(predictions)
    ground_truths = torch.cat(ground_truths)
    train_acc, train_f1 = evaluator(predictions, ground_truths)
    torch.cuda.empty_cache()

    return train_loss, 100*train_acc, 100*train_f1

def validation(model, valid_loader, criterion, device):
    model.eval()
    valid_loss = 0.0
    predictions=[]
    ground_truths = []

    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(valid_loader):
            # 將數據移動到指定的設備
            images = [img.to(device) for img in images]
            labels = labels.to(device)
            logits = model(images)
            loss = criterion(logits, labels)

            valid_loss += loss.item()
            preds = torch.argmax(logits, dim=1)

            predictions.append(preds)
            ground_truths.append(labels)

        valid_loss /= len(valid_loader)

        predictions = torch.cat(predictions)
        ground_truths = torch.cat(ground_truths)
        valid_acc, valid_f1 = evaluator(predictions, ground_truths)
        torch.cuda.empty_cache()

    return valid_loss, 100*valid_acc, 100*valid_f1



def test(model, test_loader, device):
    model.eval()
    predictions_pro = []
    predictions_stage = []

    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(test_loader):
            # 將數據移動到指定的設備
            images = [img.to(device) for img in images]
            labels = labels.to(device)

            logits = model(images)  
            pred_pro = nn.functional.softmax(logits, dim=1)
            pred_stage = torch.argmax(pred_pro, dim=1)

            predictions_pro.append(pred_pro.cpu().numpy())
            predictions_stage.append((pred_stage.cpu().numpy()))

    predictions_pro = np.concatenate(predictions_pro, axis=0)
    predictions_stage = np.concatenate(predictions_stage, axis=0)
    torch.cuda.empty_cache()

    return predictions_pro, predictions_stage



# Early Fusion model

In [ ]:

class EarlyFusion(nn.Module):
    def __init__(self, num_classes, input_size=(3, 50, 50), out_prob=True, features_grad=False, num_input_channels=66):
        super(EarlyFusion, self).__init__()

        self.out_prob = out_prob

        # VGG16 特徵提取層
        vgg16 = models.vgg16(weights='IMAGENET1K_V1', progress=True)
        vgg16.classifier = nn.Identity()

        # 修改 VGG16 的第一個卷積層的輸入通道數
        vgg16.features[0] = nn.Conv2d(num_input_channels, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

        # 固定/不固定特徵層的參數值
        for param in vgg16.features.parameters():
            param.requires_grad = features_grad
        self.backend = vgg16

        # 修改這裡的輸入維度為你的資料通道數 * VGG16 的輸出特徵大小
        self.classifier = nn.Sequential(
            nn.Linear(25088, 12544),
            nn.ReLU(),
            nn.Linear(12544, num_classes)
            
        )
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # 將所有切片的信息整合到單一的特徵表示中
        x = torch.cat(x, dim=1)

        # 使用 VGG16 特徵提取層
        x = self.backend(x)
        #print(x.size())

        # 將特徵展平
        x = x.view(x.size(0), -1)

        # 通過全連接層進行分類
        x = self.classifier(x)

        # 如果需要機率輸出，應用 softmax
        if self.out_prob:
            x = self.softmax(x)

        return x



In [ ]:
def early_fusion_modeling():

    seed_everything(config.seed)
    train_df = pd.read_csv(config.root+'/train.csv')
    test_df = pd.read_csv(config.root+'/test.csv')


    # Dataset
    print("Early fusion ")
    print("Initializing dataset...")
    dataset = MRI2DDataset(df = train_df, transforms = train_transform)
    
    print("Initializing test_dataset...")
    test_dataset = MRI2DDataset(df = test_df, transforms = train_transform)
   
    
    # split training & validation dataset 
    n = len(dataset)
    valid_size = int(n * config.valid_prob)
    train_ids , valid_ids = train_test_split(
     np.linspace(0, n - 1, n).astype("int"),
     test_size = valid_size,
     random_state = config.seed,
    )
    print(f'Number of samples in train_dataset: {Counter(dataset.labels[train_ids])}')
    print(f'Number of samples in val_dataset: {Counter(dataset.labels[valid_ids])}')
    
    # DataLoader
    train_dataset = Subset(dataset, train_ids)
    valid_dataset = Subset(dataset, valid_ids)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=config.batch_size_early, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=config.batch_size_early, shuffle=False)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=config.batch_size_early, shuffle=False)

    # settings
    print("Early fusion training")
    print("Initializing model...")
    num_classes = len(Counter(dataset.labels[train_ids]))
    model = EarlyFusion(num_classes = 2)
    model.to(config.device)
    criterion = nn.CrossEntropyLoss().to(config.device)
    optimizer = torch.optim.Adam(model.parameters(), lr = config.lr_early, weight_decay = config.weight_decay_early)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer = optimizer,
        epochs = config.epochs_early,
        steps_per_epoch = train_loader.__len__(),
        max_lr = config.lr_early,
        anneal_strategy = 'cos'
    )

    # recordings
    best_val_loss = float("inf")
    history = {
      "train": {
          "loss": [],
          "acc": [],
          "f1": []
      },
      "valid": {
          "loss": [],
          "acc": [],
          "f1": []
      },
    }
    
    for epoch in range(config.epochs_early):
        train_loss, train_acc, train_f1 = train_one_epoch(model, train_loader, optimizer, scheduler, criterion, config.device)
        valid_loss, valid_acc, valid_f1 = validation(model, valid_loader, criterion, config.device)
        
        # Log the loss and validation result
        history["train"]["loss"].append(train_loss)
        history["train"]["acc"].append(train_acc)
        history["train"]["f1"].append(train_f1)
        history["valid"]["loss"].append(valid_loss)
        history["valid"]["acc"].append(valid_acc)
        history["valid"]["f1"].append(valid_f1)

        print(f'Epoch[{epoch+1}/{config.epochs_early}], Train Loss: {train_loss:.7f}, Train Accuracy: {train_acc:.4f}%, Train F1: {train_f1:.4f}% | Valid Loss: {valid_loss:.7f}, Valid Accuracy: {valid_acc:.4f}%, Valid F1: {valid_f1:.4f}% | LR: {optimizer.state_dict()["param_groups"][0]["lr"]:.6f}')

        if valid_loss < best_val_loss:
            save_file = {
                "model": model.state_dict(),
                "optimizer": optimizer.state_dict(),
                "scheduler": scheduler.state_dict(),
                "epoch": epoch,
                "args": config
            }
            best_val_loss = valid_loss
            torch.save(save_file, "checkpoint_early.pth")
            
    best_ckpt = torch.load("checkpoint_early.pth", map_location=config.device)
    model.load_state_dict(best_ckpt["model"])

    print("Early fusion plot")
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(range(config.epochs_early), history["train"]["loss"], label='Training Loss')
    plt.plot(range(config.epochs_early), history["valid"]["loss"], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss Curves')
    plt.show()

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(range(config.epochs_early), history["train"]["acc"], label='Training Acc')
    plt.plot(range(config.epochs_early), history["valid"]["acc"], label='Validation Acc')
    plt.xlabel('Epoch')
    plt.ylabel('Acc')
    plt.legend()
    plt.title('Training and Validation Accuracy Curves')
    plt.show()

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(range(config.epochs_early), history["train"]["f1"], label='Training F1')
    plt.plot(range(config.epochs_early), history["valid"]["f1"], label='Validation F1')
    plt.xlabel('Epoch')
    plt.ylabel('F1 Score')
    plt.legend()
    plt.title('Training and Validation F1 Score Curves')
    plt.show()
    
    test_prediction_pro, test_prediction_stage = test(model, test_loader, config.device)
    # 读取现有的test.csv文件
    test_df = pd.read_csv('/kaggle/input/hw3-data/hwk03_data/hwk03_data/test.csv')

    # 创建一个DataFrame包含预测结果
    predictions = pd.DataFrame(test_prediction_pro, columns=[f'Disease({i})' for i in range(2)])
    predictions['Disease'] = test_prediction_stage
    predictions = predictions[['Disease(0)', 'Disease(1)', 'Disease']]
    # 将预测结果添加到现有DataFrame中
    test_df['Disease 0'] = predictions['Disease(0)']
    test_df['Disease 1'] = predictions['Disease(1)']
    
    test_df['Disease'] = predictions['Disease']
    # 另存为新的CSV文件
    test_df.to_csv('early.csv', index=False)
    print("early save")





In [ ]:
def main():
    
    early_fusion_modeling()
    
    
if __name__ == "__main__":
    main()